# Basic ETL

> By *Thada Srisaluakun*

---

### 1. Download Raw File

- Download `ETLDataSource1-286218-17243904498090.xlsx`
- Download `ETLDataSource2-286218-17243904776879.xlsx`

In [1]:
!gdown 1gj5-HatoF4n6V_RYB2r9vYpeawJvgf2-
!gdown 17twDjhPHlpV9ONl_Rekg_BuFLdnZLUnq

Downloading...
From: https://drive.google.com/uc?id=1gj5-HatoF4n6V_RYB2r9vYpeawJvgf2-
To: /Users/tada/Documents/GitHub/data-warehouse-mining/src/assgn01_etlAndDataProcessing/ETLDataSource1-286218-17243904498090.xlsx
100%|███████████████████████████████████████| 15.3k/15.3k [00:00<00:00, 566kB/s]
Downloading...
From: https://drive.google.com/uc?id=17twDjhPHlpV9ONl_Rekg_BuFLdnZLUnq
To: /Users/tada/Documents/GitHub/data-warehouse-mining/src/assgn01_etlAndDataProcessing/ETLDataSource2-286218-17243904776879.xlsx
100%|██████████████████████████████████████| 12.7k/12.7k [00:00<00:00, 17.9MB/s]


### 2. Load Up Data

- Import `pandas@2.2.2`
- Assign all raw dataframes

In [2]:
import pandas as pd
from pandas import DataFrame, Series

rawOrderSource1: DataFrame = pd.read_excel(io='ETLDataSource1-286218-17243904498090.xlsx', sheet_name='orderSource1')
rawProductSource1: DataFrame = pd.read_excel(io='ETLDataSource1-286218-17243904498090.xlsx', sheet_name='productSource1')
rawStateLookupSource1: DataFrame = pd.read_excel(io='ETLDataSource1-286218-17243904498090.xlsx', sheet_name='StateLookup')

rawOrderSource2: DataFrame = pd.read_excel(io='ETLDataSource2-286218-17243904776879.xlsx', sheet_name='orderSource2')
rawProductSource2: DataFrame = pd.read_excel(io='ETLDataSource2-286218-17243904776879.xlsx', sheet_name='productSource2')

### 3. Prepare Helper Dataframes

- Prepare lookup data (to be merged later)
- Function for transforming data

In [3]:
stateLookup: dict[str, str] = {
    abbreviation: stateName
    for _, stateName, abbreviation
    in rawStateLookupSource1.itertuples()
}

statusLookup: dict[int, str] = {
    1: 'Silver',
    2: 'Gold',
    3: 'Platinum',
}

def identity(x):
    return x

def getFirstName(fullName: str) -> str:
    return fullName.split()[0]

def getLastName(fullName: str) -> str:
    return fullName.split()[-1]

def removePrefixA(text: str) -> int:
    return int(text.upper().replace('A', ''))

namesSource1: DataFrame = (
    rawOrderSource1['CustomerName']
    .transform(func=[identity, getFirstName, getLastName])
    .rename(columns={'identity': 'CustomerName', 'getFirstName': 'CustomerFirstName', 'getLastName': 'CustomerLastName'})
)

cleanedOrderIdSource2: DataFrame = (
    rawOrderSource2['OrderID']
    .transform(func=[identity, removePrefixA])
    .rename(columns={'identity': 'OrderID', 'removePrefixA': 'newOrderID'})
)

totalDiscountSource2: Series = (
    (rawProductSource2['FullPrice'] - rawProductSource2['ExtendedPrice'])
    .rename(index='TotalDiscount')
)


### 4. Process Two Data Sources

- Data Source 1
  1. Join `orderSource1` with `productSource1` on `OrderID`
  2. Map abbreviation of state in `CustomerState`
  3. Split `CustomerName` to `CustomerFirstName` and `CustomerLastName`
  4. Sort columns
- Data Source 2
  1. Join `orderSource2` with `productSource2` on `OrderID`
  2. Remove `'A'` prefix in `OrderID`
  3. Map customer tier number to tier name in `CustomerStatus`
  4. Calculate `TotalDiscount`
  5. Sort columns

In [4]:
processedSource1: DataFrame = (
    rawOrderSource1
    .merge(right=rawProductSource1, how='inner', on='OrderID')

    .replace(to_replace={'CustomerState': stateLookup})

    .merge(right=namesSource1, how='inner', on='CustomerName')
    .drop(columns='CustomerName')
    
    .sort_index(axis='columns')
)

processedSource2: DataFrame = (
    rawOrderSource2
    .merge(right=rawProductSource2, how='inner', on='OrderID')

    .merge(right=cleanedOrderIdSource2, how='inner', on='OrderID')
    .drop(columns='OrderID')
    .rename(columns={'newOrderID': 'OrderID'})

    .replace(to_replace={'CustomerStatus': statusLookup})

    .drop(columns='TotalDiscount')
    .merge(right=totalDiscountSource2, left_index=True, right_index=True)
    
    .sort_index(axis='columns')
)

processedSource2.head()

,CustomerCity,CustomerFirstName,CustomerLastName,CustomerState,CustomerStatus,Discount,ExtendedPrice,FullPrice,OrderDate,OrderID,Product,ProductID,Quantity,TotalDiscount,UnitPrice
0,Kansas City,Milagros,Fehrenbach,Kansas,Gold,0.2,608.00,760.0,2023-01-11,10258,Chang,2,50,152.00,15.2
1,Kansas City,Milagros,Fehrenbach,Kansas,Gold,0.2,122.88,153.6,2023-01-11,10258,Mascarpone Fabioli,32,6,30.72,25.6
2,Kansas City,Milagros,Fehrenbach,Kansas,Gold,0.2,884.00,1105.0,2023-01-11,10258,Chef Anton's Gumbo Mix,5,65,221.00,17.0
3,Greensboro,Julio,Willard,North Carolina,Platinum,0.0,80.00,80.0,2023-01-11,10259,Sir Rodney's Scones,21,10,0.00,8.0
4,Greensboro,Julio,Willard,North Carolina,Platinum,0.0,20.80,20.8,2023-01-11,10259,Gravad lax,37,1,0.00,20.8


### 5. Combine Two Data Sources

1. Concat two processed data source
2. Combine `CustomerFirstName` and `CustomerLastName` -> `CustomerName`
3. Reset index for each rows
4. Sort columns
5. Save CSV file

In [5]:
combinedSource: DataFrame = pd.concat([processedSource1, processedSource2])
combinedSource['CustomerName'] = combinedSource['CustomerFirstName'] + ' ' + combinedSource['CustomerLastName']
combinedSource = (
    combinedSource
    .drop(columns=['CustomerFirstName', 'CustomerLastName'])
    
    .reset_index(drop=True)
    .sort_index(axis='columns')
)

combinedSource.to_excel(
    excel_writer='pandasEtl.xlsx',
    index=False,
)

combinedSource

,CustomerCity,CustomerName,CustomerState,CustomerStatus,Discount,ExtendedPrice,FullPrice,OrderDate,OrderID,Product,ProductID,Quantity,TotalDiscount,UnitPrice
0,Pittsburgh,Suzan Plock,Pennsylvania,Silver,0.00,168.00,168.0,2023-01-11,10248,Queso Cabrales,11,12,0.00,14.0
1,Pittsburgh,Suzan Plock,Pennsylvania,Silver,0.00,98.00,98.0,2023-01-11,10248,Singaporean Hokkien Fried Mee,42,10,0.00,9.8
2,Pittsburgh,Suzan Plock,Pennsylvania,Silver,0.00,174.00,174.0,2023-01-11,10248,Mozzarella di Giovanni,72,5,0.00,34.8
3,Miami,Allan Strate,Florida,Platinum,0.00,1696.00,1696.0,2023-01-11,10249,Manjimup Dried Apples,51,40,0.00,42.4
4,Miami,Allan Strate,Florida,Platinum,0.00,167.40,167.4,2023-01-11,10249,Tofu,14,9,0.00,18.6
5,Philadelphia,Elnora Willison,Pennsylvania,Platinum,0.15,1261.40,1484.0,2023-01-11,10250,Manjimup Dried Apples,51,35,222.60,42.4
6,Philadelphia,Elnora Willison,Pennsylvania,Platinum,0.00,77.00,77.0,2023-01-11,10250,Jack's New England Clam Chowder,41,10,0.00,7.7
7,Philadelphia,Elnora Willison,Pennsylvania,Platinum,0.15,214.20,252.0,2023-01-11,10250,Louisiana Fiery Hot Pepper Sauce,65,15,37.80,16.8
8,Tacoma,Daniela Becknell,Washington,Platinum,0.00,336.00,336.0,2023-01-11,10251,Louisiana Fiery Hot Pepper Sauce,65,20,0.00,16.8
9,Tacoma,Daniela Becknell,Washington,Platinum,0.05,95.76,100.8,2023-01-11,10251,Gustaf's Knäckebröd,22,6,5.04,16.8
